##  Installing Pytorch and Hugging Face libraries




In [ ]:
# Install Pytorch
%pip install "torch==2.2.2" tensorboard

# Install Hugging Face libraries
%pip install  --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"


## Logging into Huggingface

In [ ]:
!huggingface-cli login

## Reading in the data

In [ ]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score

from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)


In [ ]:
# Reading in the Airlines Twitter data
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
df = pd.read_csv('/content/drive/MyDrive/Tweets.csv' )

In [ ]:
#Classifying the positive reviews as class 2 , neutral reviews as class 1 and negative reviews as class 0 under column "sentiment"
df['sentiment'] = df['airline_sentiment'].apply(lambda rating : 2 if rating =="positive" else (1 if rating=="neutral" else 0))
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,sentiment
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),1
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),2
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),1
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),0
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),0


In [ ]:
## Renaming "text" as "Text"
df1=df.rename(columns={"text": "Text"})
df1.info()

## Splitting the data into train/val/test datasets
* train - 60%, val - 20%, test - 20%

In [ ]:
X_train, X_temp, y_train, y_temp  = train_test_split(df1['Text'],
                                                    df1['sentiment'],
                                                    test_size=0.4,
                                                    random_state=21,
                                                    stratify=df1['sentiment'])

X_val, X_test, y_val, y_test   = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=21)
df_train = pd.concat([X_train, y_train], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
print(df_train.shape, df_test.shape, df_val.shape)



(8784, 2) (2928, 2) (2928, 2)


## Convert from Pandas DataFrame to Hugging Face Dataset



In [ ]:
# Converting pandas DataFrames into Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)


In [ ]:
# Shuffle the training dataset
dataset_train_shuffled = dataset_train.shuffle(seed=21)  # Using a seed for reproducibility

In [ ]:
# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train_shuffled,
    'val': dataset_val,
    'test': dataset_test
})
dataset

## Calculating ClassWeights

In [ ]:
df_train.sentiment.value_counts(normalize=True)

,proportion
sentiment,
0,0.626935
1,0.211635
2,0.161430


In [ ]:
class_weights=(1/df_train.sentiment.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights


tensor([0.1275, 0.3776, 0.4950])

## Importing Llama 3 8B Param Pre-trained Model with 4 bit quantization

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B"

In [ ]:
## Quantization Config
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # This parameter enables 4-bit quantization, which reduces the model’s memory footprint by representing weights with 4 bits instead of the usual 16 or 32 bits.
    bnb_4bit_quant_type = 'nf4', # NormalFloat 4-bit
    bnb_4bit_use_double_quant = True, # double quantization
    bnb_4bit_compute_dtype = torch.bfloat16 # 16-bit floating-point
)


In [ ]:
## Lora Config
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=3
)

model

In [ ]:
model = prepare_model_for_kbit_training(model)
model

In [ ]:
model = get_peft_model(model, lora_config)
model

## Loading the tokenizer



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

## Updating some model configs


In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

## Measuring model performance before fine-tuning


In [ ]:
import time
start_time = time.time()

In [ ]:
# Convert reviews to a list
sentences = df_test.Text.tolist()

# Setting the batch size
batch_size = 32

# empty list to store the model outputs
all_outputs = []

# Processing the sentences in batches
for i in range(0, len(sentences), batch_size):
    # Get the batch of sentences
    batch_sentences = sentences[i:i + batch_size]

    # Tokenize the batch
    inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move tensors to the device where the model is (e.g., GPU or CPU)
    inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

    # Perform inference and store the logits
    with torch.no_grad():
        outputs = model(**inputs)
        all_outputs.append(outputs['logits'])

In [ ]:
final_outputs = torch.cat(all_outputs, dim=0)

In [ ]:
df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()

In [ ]:
def get_performance_metrics(df_test):
  y_test = df_test.sentiment
  y_pred = df_test.predictions

  print("Confusion Matrix:")
  print(confusion_matrix(y_test, y_pred))

  print("\nClassification Report:")
  print(classification_report(y_test, y_pred))

  # print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
  print("Accuracy Score:", accuracy_score(y_test, y_pred))

In [ ]:
get_performance_metrics(df_test)

Confusion Matrix:
[[1193  199  443]
 [ 349   76  195]
 [ 284   53  136]]

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.65      0.65      1835
           1       0.23      0.12      0.16       620
           2       0.18      0.29      0.22       473

    accuracy                           0.48      2928
   macro avg       0.35      0.35      0.34      2928
weighted avg       0.49      0.48      0.48      2928

Accuracy Score: 0.4798497267759563


In [ ]:
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 261.41209292411804 seconds


## Creating LLAMA tokenized dataset

In [ ]:
MAX_LEN = 512
col_to_delete = ['Text']

def llama_preprocessing_function(examples):
    return tokenizer(examples['Text'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "label")
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/8784 [00:00<?, ? examples/s]

Map:   0%|          | 0/2928 [00:00<?, ? examples/s]

Map:   0%|          | 0/2928 [00:00<?, ? examples/s]

In [ ]:
#Data Collator With Padding
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)


## Defining metrics for evaluation


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy':accuracy_score(predictions,labels)}



## Defining custom trainer with classweights


In [ ]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels and convert them to long type for cross_entropy
        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        # Extract logits assuming they are directly outputted by the model
        logits = outputs.get('logits')

        # Compute custom loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss


## Defining training args

In [ ]:
training_args = TrainingArguments(
    output_dir = 'Airline_sentiment_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 3,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

## Defining custom trainer

In [ ]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['val'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)

<ipython-input-35-1bea759fa04e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)


## Running the Trainer

In [ ]:
train_result = trainer.train()

## Prediction - Evaluating the model after fine-tuning


In [ ]:
start_time = time.time()

In [ ]:
def make_predictions(model,df_test):


  # Convert summaries to a list
  sentences = df_test.Text.tolist()

  # Define the batch size
  batch_size = 32  # You can adjust this based on your system's memory capacity

  # Initialize an empty list to store the model outputs
  all_outputs = []

  # Process the sentences in batches
  for i in range(0, len(sentences), batch_size):
      # Get the batch of sentences
      batch_sentences = sentences[i:i + batch_size]

      # Tokenize the batch
      inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

      # Move tensors to the device where the model is (e.g., GPU or CPU)
      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

      # Perform inference and store the logits
      with torch.no_grad():
          outputs = model(**inputs)
          all_outputs.append(outputs['logits'])
  final_outputs = torch.cat(all_outputs, dim=0)
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
  # df_test['predictions']=df_test['predictions'].apply(lambda l:category_map[l])


make_predictions(model,df_test)

In [ ]:
get_performance_metrics(df_test)

Confusion Matrix:
[[1605  189   41]
 [  42  478  100]
 [   8   34  431]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.87      0.92      1835
           1       0.68      0.77      0.72       620
           2       0.75      0.91      0.82       473

    accuracy                           0.86      2928
   macro avg       0.80      0.85      0.82      2928
weighted avg       0.87      0.86      0.86      2928

Accuracy Score: 0.8586065573770492


In [ ]:
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 249.56030440330505 seconds


### Saving the model trainer state and model adapters

In [ ]:
metrics = train_result.metrics
max_train_samples = len(dataset_train)
metrics["train_samples"] = min(max_train_samples, len(dataset_train))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =        3.0
  total_flos               = 41221761GF
  train_loss               =     0.4769
  train_runtime            = 2:47:21.42
  train_samples            =       8784
  train_samples_per_second =      2.624
  train_steps_per_second   =      0.328
